In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from tqdm import tqdm

In [31]:
# divece congif
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'selected device is: {device}')

selected device is: cpu


In [32]:
learning_rate = 0.01
num_epochs = 5

In [33]:
# MINST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5))
])

In [34]:
train_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=transform)

* 70,000 grayscale images of handwritten digits:

* 60,000 for training

* 10,000 for testing

* Each image is 28x28 pixels

* Digits: 0 to 9

* Labelled dataset (each image has the correct digit as a label)

In [35]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [36]:
# the CNN

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1) # output: 16x28x28
        self.pool = nn.MaxPool2d(2,2) # # output: 16x14x14
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1) # output: 32x14x14 -> pool -> 32x7x7
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10) # 10 classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        

In [37]:
model = SimpleCNN().to(device)

In [38]:
critetion = nn.CrossEntropyLoss() # no need to use softmax 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [40]:
# training loop

for epoch in range(num_epochs):
    running_loss = 0.0
    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = critetion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        loop.set_postfix(loss=loss.item())
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/5]:   0%|          | 0/938 [00:11<?, ?it/s, loss=0.164]


Epoch [1/5], Loss: 0.1476


Epoch [2/5]:   0%|          | 0/938 [00:56<?, ?it/s, loss=0.119]  

Epoch [2/5], Loss: 0.1189


Epoch [2/5]:   0%|          | 0/938 [00:56<?, ?it/s, loss=0.119]


Epoch [3/5], Loss: 0.0741


Epoch [4/5]:   0%|          | 0/938 [00:56<?, ?it/s, loss=0.0534]  

Epoch [4/5], Loss: 0.0534


Epoch [4/5]:   0%|          | 0/938 [00:56<?, ?it/s, loss=0.0534]


Epoch [5/5], Loss: 0.1948


In [41]:
# Test accuracy
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 97.81%
